### 1. 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from google.colab import drive
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 시각화 설정
sns.set_theme(style="whitegrid")
plt.rcParams['font.family'] = 'sans-serif'

drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/f1_project/data/'
file_name = 'training_data.csv'
file_path = os.path.join(base_path, file_name)

print(f"Loading data from: {file_path}")

try:
    df = pd.read_csv(file_path)
    print(f"Data Loaded Successfully: {df.shape}")
    display(df.head())
except FileNotFoundError:
    print(f"\n[Error] 파일을 찾을 수 없습니다! 경로를 확인하세요: {file_path}")
    raise  # 에러 발생 시 중단

### 2. 데이터 전처리

In [ ]:
print("\n[Step 1] Target Engineering...")

# 두 사이트의 평점이 모두 있는 경우에만 계산
if 'F1HOTorNOT' in df.columns and 'RaceFans' in df.columns:
    target_scaler = MinMaxScaler()
    df[['F1_Norm', 'RF_Norm']] = target_scaler.fit_transform(df[['F1HOTorNOT', 'RaceFans']])
    df['Excitement_Score'] = (df['F1_Norm'] + df['RF_Norm']) / 2
    print(" -> Target Created: Excitement_Score (Average of F1HOTorNOT & RaceFans)")
else:
    print(" -> Warning: Rating columns not found. Check your CSV file.")

# 타겟값(점수)이 없는 데이터(NaN) 제거
initial_len = len(df)
df = df.dropna(subset=['Excitement_Score'])
print(f" -> Dropped NaN Targets: {initial_len} -> {len(df)} rows remaining.")


# -----------------------------------------------------------------------------
# 5. Feature Selection & Data Split
# -----------------------------------------------------------------------------
print("\n[Step 2] Splitting Data (Train: 2021-2024)...")

# Feature 정의
features = [
    'Chaos_Score',          # 사고 및 혼란도 (SC/RedFlag 등)
    'Lead_Changes',         # 선두 교체 횟수
    'Gap_Std_Dev',          # 1-2위 격차 표준편차
    'Position_Gains_Total'  # 총 순위 변동 횟수
]
target = 'Excitement_Score'
group_col = 'Year'

# 2024년까지의 데이터만 학습에 사용 (Time-series logic)
train_df = df[df['Year'] <= 2024].copy()

X = train_df[features]
y = train_df[target]
groups = train_df[group_col]

print(f" -> Training Data Shape: {X.shape}")

### 3. Training

In [ ]:
# =============================================================================
# 6. Cross Validation (Custom Rolling Window by Season)
# =============================================================================

print("\n[Step 3] Cross Validation (Custom Rolling Window)...")

# 검증할 타겟 연도 리스트 (2022년부터 2024년까지)
validation_years = [2022, 2023, 2024]

model = RandomForestRegressor(n_estimators=2000, random_state=42, max_depth=5, min_samples_leaf=2)

results = []
cv_scores = []

for val_year in validation_years:
    # 1. Split (Year 기준으로 명확하게 분리)
    # 학습용: 검증 연도보다 과거인 모든 데이터
    # 검증용: 해당 검증 연도 데이터
    train_fold = df[df['Year'] < val_year]
    val_fold = df[df['Year'] == val_year]

    # 데이터가 비어있지 않은지 확인
    if len(train_fold) == 0 or len(val_fold) == 0:
        print(f"Skipping Year {val_year} (Not enough data)")
        continue

    X_tr = train_fold[features]
    y_tr = train_fold[target]

    X_val = val_fold[features]
    y_val = val_fold[target]

    # 2. Scaling (Data Leakage 방지: Train Fold로만 Fit)
    scaler = StandardScaler()
    X_tr_scaled = scaler.fit_transform(X_tr)
    X_val_scaled = scaler.transform(X_val)

    # 3. Training & Evaluation
    model.fit(X_tr_scaled, y_tr)
    y_pred = model.predict(X_val_scaled)

    r2 = r2_score(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)

    cv_scores.append(r2)
    results.append({'Val_Year': val_year, 'R2': r2, 'MAE': mae})

    # 학습 데이터 기간 표시 (예: "2021-2022")
    train_years_str = f"{train_fold['Year'].min()}-{train_fold['Year'].max()}"
    print(f" -> Train({train_years_str}) vs Val({val_year}) | R2: {r2:.4f} | MAE: {mae:.4f}")

mean_r2 = np.mean(cv_scores)
print(f"\n -> Average Validation R2 Score: {mean_r2:.4f}")

In [ ]:
print("Generating Learning Curve (Smart Way)...")

# 1. 모델 설정
rf_warm = RandomForestRegressor(n_estimators=100, warm_start=True, max_depth=5, random_state=42, n_jobs=-1, min_samples_leaf=2)

estimators_range = range(100, 2001, 100) # 10개씩 늘려가며 2000개까지
train_scores = []
val_scores = []

# 검증용 데이터 준비 (Train/Val 나누기)
# 시각화를 위해 Train 데이터 중 일부를 떼어내거나, 마지막 Fold 데이터를 사용
vis_train_mask = df['Year'] <= 2023
vis_val_mask = df['Year'] == 2024

X_vis_tr = df[vis_train_mask][features]
y_vis_tr = df[vis_train_mask][target]
X_vis_val = df[vis_val_mask][features]
y_vis_val = df[vis_val_mask][target]

# 스케일링 (시각화용)
scaler_vis = StandardScaler()
X_vis_tr_scaled = scaler_vis.fit_transform(X_vis_tr)
X_vis_val_scaled = scaler_vis.transform(X_vis_val)

# 2. 루프를 돌며 나무를 추가 (Epoch와 유사한 효과)
for n in estimators_range:
    rf_warm.n_estimators = n # 나무 개수 업데이트
    rf_warm.fit(X_vis_tr_scaled, y_vis_tr) # 기존 나무 유지하고 추가 학습

    # 점수 기록
    train_scores.append(r2_score(y_vis_tr, rf_warm.predict(X_vis_tr_scaled)))
    val_scores.append(r2_score(y_vis_val, rf_warm.predict(X_vis_val_scaled)))

# 3. 그래프 그리기
plt.figure(figsize=(10, 6))
plt.plot(estimators_range, train_scores, '-', label='Train R2', color='blue')
plt.plot(estimators_range, val_scores, '-', label='Validation R2', color='orange')

plt.title('Learning Curve: Performance over n_estimators (Warm Start)')
plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('R2 Score')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# -----------------------------------------------------------------------------
# 7. Final Model Training & Saving
# -----------------------------------------------------------------------------
print("\n[Step 4] Final Training & Saving...")

# 1. 전체 Train Set(2021-2024)으로 Scaler 학습
final_scaler = StandardScaler()
X_final_scaled = final_scaler.fit_transform(X)

# 2. 전체 Train Set으로 모델 학습
final_model = RandomForestRegressor(n_estimators=2000, random_state=42, max_depth=5, min_samples_leaf=2)
final_model.fit(X_final_scaled, y)

# 3. 모델과 스케일러 저장 (evaluation/inference 단계에서 필수)
model_save_path = os.path.join(base_path, 'f1_excitement_model.pkl')
scaler_save_path = os.path.join(base_path, 'scaler_features.pkl')

joblib.dump(final_model, model_save_path)
joblib.dump(final_scaler, scaler_save_path)

print(f" -> Model saved at: {model_save_path}")
print(f" -> Scaler saved at: {scaler_save_path}")
print("\n[Success] Training process completed.")

## Summary

### 1. 모델 설계의 적절성
- 데이터의 특성과 복잡한 상호작용을 고려하여 **Random Forest Regressor**를 채택했습니다.
- 과적합 방지와 안정적인 예측을 위해 앙상블 기법을 사용했으며, 데이터셋 크기가 작음을 고려하여 딥러닝보다는 머신러닝 모델이 적합하다고 판단했습니다.

### 2. 데이터셋 분할의 명확성
- 시계열 데이터(F1 시즌)의 인과성을 보존하기 위해 **Rolling Window (Time-series Split)** 방식을 적용했습니다.
- **Ratio & Period**:
    - **Train**: 2021 ~ 2023 시즌 (학습)
    - **Validation**: 2024 시즌 (검증 및 튜닝)
    - **Final Test**: 2025 시즌 (최종 평가 - **Data Leakage 차단**을 위해 별도 파일로 분리하여 `evaluation.ipynb`에서 수행)

### 3. 학습 과정의 투명성 및 시각화
- Random Forest에는 Epoch 개념이 없으므로, **`n_estimators`를 증가**시키며 성능 변화를 추적했습니다.
- `warm_start=True` 옵션을 활용하여 **Learning Curve** (Train R2 vs Validation R2)를 시각화했습니다.
    - 이를 통해 모델이 초기에는 데이터 부족으로 과소적합되다가, 데이터가 누적될수록 성능이 향상됨을 확인했습니다.
- **Analysis**: 학습 데이터에 대한 R2(0.65 이상)와 검증 데이터에 대한 R2(0.35) 차이를 통해 모델의 과적합 경향과 데이터 패턴의 변화를 분석했습니다.

### 4. Validation 성능 모니터링
- 회귀 문제에 적합한 **R2 Score**와 **MAE** (Mean Absolute Error)를 사용했습니다.
- **Results**: 연도별 검증(Rolling Window)을 통해 2022년(규정 대격변)의 성능 저하와 2024년(데이터 누적 후)의 성능 향상을 정량적으로 확인했습니다.

### 5. 재현 가능성
- 학습된 모델(`f1_excitement_model.pkl`)과 전처리 스케일러(`scaler_features.pkl`)를 `joblib`을 통해 저장했습니다.
- 별도의 `inference.ipynb` 및 `evaluation.ipynb`에서 저장된 파일을 로드하여 동일한 성능을 재현할 수 있도록 구성했습니다.